In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [3]:
train_df = pd.read_csv('Untitled Folder/train_crime.csv' , parse_dates = ['Dates'])
test_df = pd.read_csv('Untitled Folder/test_crime.csv' , parse_dates = ['Dates'])

In [4]:
train_df.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [5]:
len(test_df.Dates.dt.year.unique())

13

In [6]:
len(train_df.Dates.dt.year.unique())

13

In [7]:
#removing target
encoder = LabelEncoder()
target = encoder.fit_transform(train_df.Category)

In [8]:
#for training set ( 35 features)
district_train = pd.get_dummies(train_df.PdDistrict)

day_train = pd.get_dummies(train_df.DayOfWeek)

year_train = train_df.Dates.dt.year
year_train = pd.get_dummies(year_train)

hour_train = train_df.Dates.dt.hour
month_train = train_df.Dates.dt.month

encoder_1 = LabelEncoder()
Address_train = encoder_1.fit_transform(train_df.Address)

train_final = pd.concat( [district_train , day_train ,year_train ] , axis = 1)

train_final['Address'] = Address_train
train_final['Hour'] = hour_train
train_final['Month'] = month_train
train_final['Lat'] = train_df['X']
train_final['Lon'] = train_df['Y']

In [9]:
#for test set (35 features)
district_test = pd.get_dummies(test_df.PdDistrict)

day_test = pd.get_dummies(test_df.DayOfWeek)

year_test = test_df.Dates.dt.year
year_test = pd.get_dummies(year_test)

hour_test = test_df.Dates.dt.hour
month_test = test_df.Dates.dt.month

encoder_2 = LabelEncoder()
Address_test = encoder_2.fit_transform(test_df.Address)

test_final = pd.concat( [district_test , day_test ,year_test ] , axis = 1)

test_final['Address'] = Address_test
test_final['Hour'] = hour_test
test_final['Month'] = month_test
test_final['Lat'] = test_df['X']
test_final['Lon'] = test_df['Y']

In [10]:
test_final.shape

(884262, 35)

In [11]:
train_final.shape

(878049, 35)

In [12]:
X_train , X_test , y_train , y_test = train_test_split(train_final ,  target , train_size = 0.8)

C:\Users\Husnain\Downloads\ANACONDA\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [13]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import log_loss

clf = BernoulliNB().fit(X_train , y_train)
#clf.score(X_test , y_test)
predicted_probs = clf.predict_proba(X_test)
log_loss(y_test, predicted_probs) 

2.5931401761995585

In [14]:
encoder.classes_

array(['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE',
       'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION',
       'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING',
       'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING',
       'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES',
       'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE',
       'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE',
       'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE',
       'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT',
       'WARRANTS', 'WEAPON LAWS'], dtype=object)

In [26]:
predicted_results = clf.predict_proba(test_final)
answer = pd.DataFrame( predicted_results , columns = encoder.classes_)

In [27]:
ID = test_df.index

In [28]:
answer['ID'] = ID
#answer = answer.reset_index(ID)

In [32]:
answer.head()

,ID,ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,...,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS
0,0,0.007781,0.132046,0.000019,0.001532,0.035770,0.001068,0.002433,0.016925,0.003015,...,0.000101,0.004694,0.000216,0.040415,1.750919e-06,0.006200,0.062693,0.074171,0.040383,0.024086
1,1,0.007781,0.132046,0.000019,0.001532,0.035770,0.001068,0.002433,0.016925,0.003015,...,0.000101,0.004694,0.000216,0.040415,1.750919e-06,0.006200,0.062693,0.074171,0.040383,0.024086
2,2,0.001626,0.084315,0.000029,0.000317,0.043842,0.001732,0.002624,0.013178,0.003676,...,0.000036,0.006846,0.000270,0.026191,7.270436e-07,0.005858,0.049018,0.049620,0.033969,0.007988
3,3,0.003147,0.126376,0.000020,0.001515,0.033651,0.000884,0.003101,0.009742,0.002698,...,0.000087,0.004192,0.000373,0.036335,4.375381e-07,0.004278,0.071247,0.106465,0.025458,0.016946
4,4,0.003147,0.126376,0.000020,0.001515,0.033651,0.000884,0.003101,0.009742,0.002698,...,0.000087,0.004192,0.000373,0.036335,4.375381e-07,0.004278,0.071247,0.106465,0.025458,0.016946


In [30]:
answer = answer[['ID' ,'ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY',
       'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE', 'DRUG/NARCOTIC',
       'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION', 'FAMILY OFFENSES',
       'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING', 'KIDNAPPING',
       'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON',
       'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY/OBSCENE MAT',
       'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY', 'RUNAWAY',
       'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE',
       'STOLEN PROPERTY', 'SUICIDE', 'SUSPICIOUS OCC', 'TREA', 'TRESPASS',
       'VANDALISM', 'VEHICLE THEFT', 'WARRANTS', 'WEAPON LAWS']]
#answer.set_index( 'ID' , inplace = True)
#answer.columns

In [33]:
answer.to_csv('kaggle_crime2.csv', index = False , index_label = 'Id' )
